In [7]:
import airflow
from airflow import DAG
from datetime import datetime
import os

In [12]:
from airflow.operators.python import PythonOperator

In [53]:
dag_path = os.getcwd()

In [3]:
dag = DAG(dag_id='synthetic_data',
          start_date=datetime(2023,3,12),
          schedule='*/1 * * * *'
         )

In [7]:
dag.filepath

C:\Users\visha\AppData\Local\Temp\ipykernel_10028\3582464681.py:1 RemovedInAirflow3Warning: filepath is deprecated, use relative_fileloc instead

'C:\\Users\\visha\\AppData\\Local\\Temp\\ipykernel_10028\\3030468319.py'

In [11]:
def create_synthetic_data():
    from faker import Faker
    from google.cloud import storage
    from google.cloud import aiplatform
    import os,datetime

    os.environ['GOOGLE_APPLICATION_CREDENTIALS']= "L:\\gcp-learning-feb-2023-5e0385a3a263.json"

    PROJECT_ID = "gcp-learning-feb-2023"
    REGION     = 'us-central1'
    BUCKET     = 'gcp-bucket-19feb2023'
    BUCKET_URI = 'gs://gcp-bucket-19feb2023/apache_airflow'

    fake = Faker()
    storage_client = storage.Client(project=PROJECT_ID)
    bucket = storage_client.bucket(bucket_name=BUCKET)
    blob = bucket.blob('feature-store-data/cc_data.csv')
    #Creating the Synthetic Data and Storing it in the GCS Bucket
    with blob.open(mode='w') as fl:
        print("Start Time is : ",datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ"))
        for n in range(1000000):
            if n==0:
                header = 'cc_num,cust_num,credit_limit,zip_code,cash_back_pct,ftr_dttm\n'
                fl.write(header)
            else:
                cc_num        = fake.credit_card_number()
                cust_num      = fake.uuid4()
                credit_limit  = fake.numerify(text='%#000')
                zip_code      = fake.zipcode()
                cash_back_pct = fake.numerify(text='%.#')
                ftr_dttm      = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")
                cc_row        = cc_num+','+cust_num+','+credit_limit+','+zip_code+','+cash_back_pct+','+ftr_dttm+'\n'
                #print(cc_row)
                fl.write(cc_row)
        print("End Time is : ",datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ"))    

In [ ]:
def load_csv_to_bq():
    

In [12]:
def create_instance_list():
    from google.cloud import storage
    import os
    import pandas as pd
    os.environ['GOOGLE_APPLICATION_CREDENTIALS']= "L:\\gcp-learning-feb-2023-5e0385a3a263.json"
    PROJECT_ID = "gcp-learning-feb-2023"
    REGION     = 'us-central1'
    BUCKET     = 'gcp-bucket-19feb2023'
    BUCKET_URI = 'gs://gcp-bucket-19feb2023/apache_airflow'
    storage_client = storage.Client(project=PROJECT_ID)
    bucket = storage_client.bucket(bucket_name=BUCKET)
    blob = bucket.blob('apache_airflow/cc_data.csv')
    blob.download_to_filename('cc_data_from_gcs.csv')
    df = pd.read_csv('cc_data_from_gcs.csv',dtype={'cc_num':'str'})
    df2 = df[['cc_num','ftr_dttm']]
    #Created Read Instance List for Batch Serving to BQ
    df2.to_csv('cc_read_instance_list.csv',index=False,header=['creditcard_dtl','timestamp'])
    blob2 = bucket.blob('feature-store-data/cc_read_instance_list.csv')
    #Uploaded Read Instance Lst to GCS Bucket
    blob2.upload_from_filename('cc_read_instance_list.csv')

In [13]:
create_synthetic_data = PythonOperator(task_id='create_synthetic_data',
                                       dag=dag,
                                       python_callable=create_synthetic_data
                                      )

In [14]:
create_instance_list = PythonOperator(task_id='create_instance_list',
                                      dag=dag,
                                      python_callable=create_instance_list
                                     )

In [15]:
create_synthetic_data >> create_instance_list

<Task(PythonOperator): create_instance_list>

In [ ]:
######################################################################################################

In [1]:
import json

In [3]:
fl = open('D:\\Airflow_Docker\\gcp_basic_details.json','r')

In [4]:
df = json.load(fl)

In [6]:
df['PROJECT_ID']

'gcp-learning-feb-2023'

In [10]:
def read_json(ti):
    fl = open('D:\\Airflow_Docker\\gcp_basic_details.json','r')
    df = json.load(fl)
    ti.xcom_push(key="PROJECT_ID",value=df['PROJECT_ID'])
    ti.xcom_push(key="BUCKET",value=df['BUCKET'])

In [11]:
def display_json(ti):
    PROJECT_ID = ti.xcom_pull(key="PROJECT_ID",task_ids="read_json")
    BUCKET = ti.xcom_pull(key="BUCKET",task_ids="read_json")
    print(PROJECT_ID +'/'+BUCKET)

In [13]:
with open('abc.txt',mode='w') as fl:
    header = 'cc_num,cust_num,credit_limit,zip_code,cash_back_pct,ftr_dttm\n'
    fl.write(header)
        

In [ ]:
pip list

In [9]:
import faker

In [10]:
import Faker

ModuleNotFoundError: No module named 'Faker'

In [11]:
pip install Faker

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install google-cloud

Note: you may need to restart the kernel to use updated packages.
